<a href="https://colab.research.google.com/github/Vaibhav-Bhatia7920/Vit-RNN-Video-Classifier/blob/main/Vit-RNN%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
from IPython.display import Image

In [2]:
IMG_SIZE = 384
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
os.chdir("/content/drive/MyDrive/UCF101")
print(os.getcwd())

/content/drive/MyDrive/UCF101


In [3]:
train_file = os.path.join(os.getcwd(),"train.csv")
test_file = os.path.join(os.getcwd(),"test.csv")
train_df = pd.read_csv(train_file)

test_df = pd.read_csv(test_file)
print(pd.unique(train_df["tag"]))
print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

['CricketShot' 'PlayingCello' 'Punch' 'ShavingBeard' 'TennisSwing']
Total videos for training: 594
Total videos for testing: 224


,video_name,tag
463,v_ShavingBeard_g23_c07.avi,ShavingBeard
64,v_CricketShot_g17_c02.avi,CricketShot
76,v_CricketShot_g19_c03.avi,CricketShot
329,v_Punch_g21_c06.avi,Punch
232,v_PlayingCello_g25_c02.avi,PlayingCello
495,v_TennisSwing_g10_c06.avi,TennisSwing
128,v_PlayingCello_g09_c05.avi,PlayingCello
592,v_TennisSwing_g25_c05.avi,TennisSwing
205,v_PlayingCello_g21_c01.avi,PlayingCello
362,v_ShavingBeard_g08_c04.avi,ShavingBeard


In [4]:
import cv2
import numpy as np
import torch



def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y: start_y + min_dim, start_x: start_x + min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    a = 0

    # Check if GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device}")

    try:
        while True:
            ret, frame = cap.read()
            if not ret and a % 2 != 0:
                break

            # Move frame to GPU
            frame = torch.tensor(frame, dtype=torch.float32, device=device)
            frame = frame.permute(2, 0, 1)  # Change the channel order

            # Perform GPU operations
            frame = torch.nn.functional.interpolate(frame.unsqueeze(0), size=resize, mode='bilinear', align_corners=False)
            frame = frame.squeeze(0).permute(1, 2, 0).to('cpu').numpy().astype(np.uint8)

            # Perform CPU operations
            frame = crop_center_square(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            a = a + 1

            if len(frames) == max_frames:
                break

    finally:
        cap.release()
        return np.array(frames)

# Example usage:
# frames = load_video("path/to/video.mp4", max_frames=10)


In [5]:
!pip install timm
import torch
import torch.nn as nn
from timm import create_model

def ensure_available_device():
    """Ensure that only GPU is used if available, otherwise use CPU."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device}.")
    return device

class Identity(nn.Module):
    def forward(self, x):
        return x

def create_feature_extractor(model_name='vit_tiny_patch16_384', device='gpu'):

    model = create_model(model_name, pretrained=True)

    # Remove last classification layer
    model.head = nn.Identity()

    # Access the penultimate layer's output
    model.norm = Identity()

    model.eval()
    device = ensure_available_device()
    model = model.to(device)

    def feature_extractor(images):
        features = []
        for img_tensor in images:
            img_tensor = img_tensor.reshape([3, 384, 384])
            img_tensor = img_tensor.to(device)
            output = model(img_tensor.unsqueeze(0))
            feature = output.detach().cpu().numpy()
            feature = feature.reshape([192])
            features.append(feature)

        return torch.tensor(features).to(device)

    return feature_extractor


device = "cuda"

batch_of_images = torch.rand([30, 3, 384, 384]).to(device)


model_name = 'vit_tiny_patch16_384'


extractor = create_feature_extractor(model_name)


features = extractor(batch_of_images)
print(features.shape)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

Using cuda.
torch.Size([30, 192])


<ipython-input-5-12eb3fba989b>:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(features).to(device)


In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import torch

train_labels = train_df["tag"].tolist()
vocab_np = LabelEncoder().fit_transform(train_labels)
vocab = torch.tensor(vocab_np, dtype=torch.long, device='cuda')
print(vocab)


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [7]:


test_labels = test_df["tag"].tolist()
vocab_np_2 = LabelEncoder().fit_transform(test_labels)
vocab_2 = torch.tensor(vocab_np_2, dtype=torch.long, device='cuda')
print(vocab_2)


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0')


In [8]:
import torch
import os

def extract_features(folder_path):
    train_features = []

    video_files = [file for file in os.listdir(folder_path) if file.endswith(".avi")]
    a = 0

    model_name = 'vit_tiny_patch16_384'
    feature_extractor = create_feature_extractor(model_name)

    for vid in video_files:
        path = os.path.join(folder_path, vid)
        framed = load_video(path, 30)
        framed = framed.reshape([30, 3, 384, -1])
        print(framed.shape)
        framed = torch.tensor(framed).to(dtype=torch.float32).to('cuda')

        a = a + 1
        feat = feature_extractor(framed)
        print(a)
        print(feat.shape)

        train_features.append(feat)

    train_features_tensor = torch.stack(train_features, dim=0)

    return train_features_tensor

os.chdir("/content/drive/MyDrive/UCF101/train")
print(os.getcwd())
train_folder_path = os.getcwd()
print(train_folder_path)

train_features_tensor = extract_features(train_folder_path)

print(train_features_tensor.shape)


/content/drive/MyDrive/UCF101/train
/content/drive/MyDrive/UCF101/train
Using cuda.
Using cuda
(30, 3, 384, 384)
1
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
2
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
3
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
4
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
5
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
6
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
7
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
8
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
9
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
10
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
11
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
12
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
13
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
14
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
15
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
16
torch.Size([30, 192])
Using cuda
(30, 3, 384, 384)
17
torch.Size([30, 192])
Using c

In [9]:
import torch
import os

def extract_features(folder_path):
    test_features = []

    video_files = [file for file in os.listdir(folder_path) if file.endswith(".avi")]
    a = 0

    model_name = 'vit_tiny_patch16_384'
    feature_extractor = create_feature_extractor(model_name)

    for vid in video_files:
        path = os.path.join(folder_path, vid)
        framed = load_video(path, 30)
        framed = framed.reshape([30, 3, 384, -1])
        framed = torch.tensor(framed).to(dtype=torch.float32).to('cuda')

        a = a + 1
        feat = feature_extractor(framed)

        test_features.append(feat)

    test_features_tensor = torch.stack(test_features, dim=0)

    return test_features_tensor

os.chdir("/content/drive/MyDrive/UCF101/test")
test_folder_path = os.getcwd()

test_features_tensor = extract_features(test_folder_path)

print(test_features_tensor.shape)


Using cuda.
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cuda
Using cud

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


class SequenceModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SequenceModel, self).__init__()
        self.gru1 = nn.GRU(num_features, 16, batch_first=True)
        self.gru2 = nn.GRU(16, 8, batch_first=True)
        self.dropout = nn.Dropout(0.4)
        self.dense1 = nn.Linear(8, 8)
        self.dense2 = nn.Linear(8, num_classes)
        for param in self.parameters():
            param.requires_grad = True

    def forward(self, x):
        _, h = self.gru1(x)
        h = h[-1].unsqueeze(0)
        h = self.gru2(h)[1][-1]
        x = self.dropout(h)
        x = self.dense1(x)
        x = self.dense2(x)

        return x


def run_experiment(model, train_data, train_labels, test_data, test_labels,temp, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    train_dataset = TensorDataset(train_data, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)

    test_dataset = TensorDataset(test_data, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=30, shuffle=False)
    for epoch in range(epochs):
        lossy = 0
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = torch.stack([model(i) for i in inputs])
            outputs = outputs.type(torch.float32)
            labels = labels.type(torch.float32)
            labels.requires_grad =True
            loss = criterion(outputs, labels.long())  # Ensure labels are of type long
            lossy  = lossy +loss.item()
            loss.backward()
            optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = torch.stack([model(i) for i in inputs])
                outputs = outputs.type(torch.float32)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                predicted = predicted.type(torch.float32)
                labels = labels.type(torch.float32)
                loss1 = nn.MSELoss()(predicted,labels)
        accuracy = correct / total
        print(f"Epoch {epoch + 1}/{epochs}, Test Accuracy: {accuracy * 100:.2f}% and the loss is {lossy/30}")

    return model

# Example usage:

train_data = torch.tensor(train_features_tensor)
test_data = torch.tensor(test_features_tensor)
num_features = train_data.size(2)
num_classes = 5

sequence_model = SequenceModel(num_features, num_classes)

trained_model = run_experiment(sequence_model, train_data, vocab, test_data, vocab_2,0, epochs=50)


<ipython-input-10-b19d113ac435>:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data = torch.tensor(train_features_tensor)
<ipython-input-10-b19d113ac435>:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = torch.tensor(test_features_tensor)


Epoch 1/50, Test Accuracy: 29.91% and the loss is 0.9569371342658997
Epoch 2/50, Test Accuracy: 30.80% and the loss is 0.7568450748920441
Epoch 3/50, Test Accuracy: 44.20% and the loss is 0.6902911484241485
Epoch 4/50, Test Accuracy: 52.68% and the loss is 0.6316227336724599
Epoch 5/50, Test Accuracy: 42.41% and the loss is 0.6118910034497579
Epoch 6/50, Test Accuracy: 50.89% and the loss is 0.5646131197611491
Epoch 7/50, Test Accuracy: 47.32% and the loss is 0.5830970267454784
Epoch 8/50, Test Accuracy: 40.62% and the loss is 0.5545182923475901
Epoch 9/50, Test Accuracy: 45.98% and the loss is 0.47762257158756255
Epoch 10/50, Test Accuracy: 51.34% and the loss is 0.43845584193865456
Epoch 11/50, Test Accuracy: 45.09% and the loss is 0.4487963298956553
Epoch 12/50, Test Accuracy: 37.50% and the loss is 0.41385432283083595
Epoch 13/50, Test Accuracy: 39.29% and the loss is 0.39567379653453827
Epoch 14/50, Test Accuracy: 39.73% and the loss is 0.40094578862190244
Epoch 15/50, Test Accura